<a href="https://www.dask.org/" target="_blank">
<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">
</a>

# Parallel programming

In this notebook, we will explain how Dask enables the parallelization of basic sequential programs throught domain decomposition (data parallelism) and functional decomposition (task parallelism).

**Introduction**

Dask is a **library for parallel/distributed computing** in Python. 

* Python users can achieve **data parallelism**, by using Dask libraries such as `Dask Array` (mimic Numpy), `Dask DataFrame` (mimic Pandas), or `Dask Bag` (for large lists).
* Users can also achieve **task parallelism** by using libraries such as `Dask Delayed`.

**Content**

1. What is parallel programming
2. Problem decomposition strategies
3. Example

**Learning outcomes**
* Define parallel programmming
* Describe parallel programmming most common problem decomposition strategies.
* Identify which Dask data structures enable domain and functional decomposition.
* Exemplifies in which basic cases it is not worth parallelizing.

## 1. What is parallel programming

Parallel programmming is the use of two or more processor/cores in combination to solve a single problem.

In order to use two or more processor/cores **we must divivde the problem**.

Parallel programming is about **divide and conquer**, that is partitioning the problem in order to feed this partitions (chunks) into multiple processors in a single machine or a distributed system. Then processors can perform computations on these partitions at the same time.

| Sequential programming                                                                                        | Parallel programming                                                                                        |
|---------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| The problem was **NOT** devided by the programmer                                                            | The problem was divided by the programmer                                                                   |
| <img src="https://raw.githubusercontent.com/DonAurelio/dask-tutorial-2023/main/img/sequential_programming.png"> | <img src="https://raw.githubusercontent.com/DonAurelio/dask-tutorial-2023/main/img/parallel_programming.png"> |

# 2. Problem decomposition strategies

Problem decomposition (or division) is how parallel computing is achieved. Two decomposition strateties dominate when dealing with scientific and data analysis computations: domain and functional decomposition. 

**Domain decomposition or Data parallelism**

* The main idea is to **partition the domain of the problem**. In scientific computing, the problem domain is commonly represented via **vectors** and **matrices** whilst in data analystics, the problem domain is commonly represented via **data tables**.
* Implies partitioning data to processes such that *a single portion of data is assigned to a single core* [1].
* Implies the simulataneous execution of the same function (operation) across the elements of a dataset [2].

**Figure:** *[Left]* The domain is partitioned into chunks. Chucks are represenetd by colors. Every chunk is feed into a single core. *[Right]* Chunks in the figure are represented by subtables. Every subtable is feed into a single core.

| Array                                                                                                                             | Table                                                                                                                             |
|-----------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------|
|                                                         |
| <img src="https://raw.githubusercontent.com/DonAurelio/dask-tutorial-2023/main/img/domain_decomposition_array.png" width="600px"> | <img src="https://raw.githubusercontent.com/DonAurelio/dask-tutorial-2023/main/img/domain_decomposition_table.png" width="600px"> |


**Functional decomposition or Task parallelism** 

* The main idea is to **divide the code** used to solve a problem. This is commonly acheved by identifing functions, operations or secctions of the code that are independent of others.
* Implies partitioning code to processes such that *a single portion of code (e.g., a function) is assigned to a single core* [1].
* Implies the simultaneous execution of multiple and different functions across the same or different data sets [2].

**Figure:** *[Left]* The code represents the following ecuation $z = x^2 + y^2$ for $x=2$ and $y=4$. *[Right]* If the program is executed in a sequential approach, it will take 4 time steps. However, when considering functional decomposition, the same function `squared` can be applied in parallel to $2$ and $4$ since these values are independent. Then, the amount of time steps is reduced to 3.

<center>
<img src="https://raw.githubusercontent.com/DonAurelio/dask-tutorial-2023/main/img/functional_decomposition.png">
</center>

## 3. Example

**Problem**. Consider the following equation for arrays of shape `(10000, 10000)`. Suppose $x$ and $y$ are both matrices of ones.

$$
  z = x^2 + y^2
$$

We will solve the above mentioned problem using the following approaches:

1. Sequential porgrammming
2. Parallel programming: Automatic parallelization
3. Parallel programming: Domain decomposition
4. Parallel programming: Functional decomposition

A the end of this notebook, you should be able to provide answers to the following questions:

1. How many cores are we using when performing sequential programming?
2. How many cores we are able to use with parallel programming?
3. Which Dask data structure (Dask Array or Dask Delayed) enables domain decompositon for parallelziation?
4. Which Dask data structure (Dask Array or Dask Delayed) enables functional decompositon for parallelization?
5. Which parallelization strategy is the best (automatic paralleization, functional or domain decomposition) when considering the implemented algorithms?

### Sequential programming approach

Open the file [matrix_sum_python.py](source/1/0.matrix_sum_python.py).

In [ ]:
!srun -N 1 -c 1 time -f 'Time: %e seconds \nCPU usage: %P \nMem usage: %M kbytes' \
python3.10 source/1/0.matrix_sum_python.py

### Parallel programming approach: Automatic parallelization

Open the file [matrix_sum_numpy.py](source/1/1.matrix_sum_numpy.py).

In [ ]:
!srun -N 1 --exclusive time -f 'Time: %e seconds \nCPU usage: %P \nMem usage: %M kbytes' \
python3.10 source/1/1.matrix_sum_numpy.py

### Parallel programming approach: Domain decomposition

In order to achieve parallelism, using the domain decomposition approach, we will use `Dask Array`. 

__1. Import required libraries, define required variables and functions__

In [ ]:
import dask.array as da

shape = (10000,10000)
chunks = (100,100)

__2. Create the first array__ using `Dask Array` and perform domain decomposition. The `chucks` param tells Dask the size of every chuck in the decomposition. 

_Hint: Play with the `shape` and `chunks` params, observe the number of chucks created._

In [ ]:
x = da.ones(shape=shape, chunks=chunks)
x

__3. Create the second array__ using `Dask Array` and perform domain decomposition.

In [ ]:
y = da.ones(shape=shape, chunks=chunks)
y

__3. Write the parallel version of the program__

*Hint: you just need to write the equation $z = x^2 + y^2$ in Python langauge*

In [ ]:
z = (x**2) + (y**2)
z

__3. Visualize the computations.__

_Hint: These computations will be performed in all the cores available in your computer._

__WARNING:__ _Reduce the amount of the chunks to 10 or less in order to be able to visulalize. If you do not do that it will take your whole life waiting..._

In [ ]:
z.visualize()

**5. Compute the result** of the equation.

Open the file [matrix_sum_dask_array.py](source/1/2.matrix_sum_dask_array.py).

In [ ]:
!srun -N 1 --exclusive time -f 'Time: %e seconds \nCPU usage: %P \nMem usage: %M kbytes' \
python3.10 source/1/2.matrix_sum_dask_array.py

### Parallel programming approach: Functional decomposition

In order to achieve parallelism, using the functional decomposition approach, we will use `Dask Delayed`. However, **we need first to adjust our code** to be able to apply functional decomposition.

__1. Import required libraries, define required variables and functions__

**Remember the problem**. Consider the following equation for arrays of shape `(10000, 10000)`. Suppose $x$ and $y$ are both matrices of ones.

$$
  z = x^2 + y^2
$$

In [ ]:
import numpy as np
import dask

shape = (10000,10000)

def squared(a):
    return a**2

def add(a,b):
    return a + b

__2. Create the arrays__

In [ ]:
x = np.ones(shape=shape)
y = np.ones(shape=shape)

__3. Write the equation in terms of functions__

_Hint: you just need to write the equation $z = x^2 + y^2$ in Python langauge, but using the functions `squared` and `add` in place of `**` and `+`._

In [ ]:
%%time 

a = squared(x)
b = squared(y) 
c = add(a,b)
c

__4. Parallelize the previous program__ using Dask Delayed.

In [ ]:
a = dask.delayed(squared)(x)
b = dask.delayed(squared)(y) 
c = dask.delayed(add)(a,b)

__5. Visualize the computations__

In [ ]:
c.visualize()

__6. Execute the computation__

Open the file [matrix_sum_dask_delayed.py](source/1/3.matrix_sum_dask_delayed.py).

In [ ]:
!srun -N 1 --exclusive time -f 'Time: %e seconds \nCPU usage: %P \nMem usage: %M kbytes' \
python3.10 source/1/3.matrix_sum_dask_delayed.py

# [Excerise 1](labs/Lab1.ipynb)

# References

1. Vitorović, A., Tomašević, M. V., & Milutinović, V. M. (2014). Manual parallelization versus state-of-the-art parallelization techniques: The spec cpu2006 as a case study. In Advances in Computers (Vol. 92, pp. 203-251). Elsevier.
2. Terrell, R. (2018). Concurrency in. NET: Modern patterns of concurrent and parallel programming. Simon and Schuster.